<a href="https://colab.research.google.com/github/sidharthdk/BMS-Data-pre-processing-files/blob/main/SNL_NCA_BATTERY_DATASET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# === SNL NCA BATTERY DATASET: LOCAL ZIP → CLEANED ZIP ===
# Upload your SNL NCA ZIP to Colab Files panel, then RUN

from google.colab import files
import zipfile
import pandas as pd
import numpy as np
from pathlib import Path

# Step 1: Upload ZIP file
print("📤 Upload your SNL NCA battery dataset ZIP:")
uploaded = files.upload()
zip_path = list(uploaded.keys())[0]
print(f"✅ Processing: {zip_path}")

# Step 2: Extract to temp directory
raw_dir = Path("/content/snl_nca_raw")
raw_dir.mkdir(exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall(raw_dir)
print(f"✅ Extracted to {raw_dir}")

csv_files = list(raw_dir.rglob("*.csv"))
print(f"Found {len(csv_files)} CSV files")

# Step 3: SNL NCA config (18650 NCA cells, 2.7-4.4V typical SNL range)[web:73][web:82]
CONFIG = {
    'interp_gap': 12,        # Slightly higher for NCA discharge plateaus
    'v_min': 2.7,            # NCA lower cutoff
    'v_max': 4.4,            # NCA upper cutoff (SNL high-voltage tests)
    'drop_bad_cycles': True,
    'capacity_max': 3.0      # Typical SNL NCA ~2.5-3Ah
}

clean_dir = Path("/content/snl_nca_cleaned")
clean_dir.mkdir(exist_ok=True)

def clean_snl_nca_file(path: Path) -> bool:
    """Clean single SNL NCA file (cycle_data or timeseries)"""
    fname = path.name
    print(f"  Cleaning {fname}...")
    try:
        df = pd.read_csv(path)
        if df.empty:
            print("    ⚠ Empty file, skipped")
            return False
        orig_rows = len(df)

        # 1) SNL column standardization
        df.columns = (
            df.columns.str.strip().str.lower()
            .str.replace(r'[^a-z0-9_]', '', regex=True)
            .str.replace(r'_+', '_', regex=True)
        )

        # Core column mapping (BatteryML/SNL format)
        rename_map = {
            'cycle': 'cycle_index', 'cycleid': 'cycle_index', 'cycle_number': 'cycle_index',
            'test_time_s': 'test_time_s', 'time_s': 'test_time_s', 'time': 'test_time_s',
            'voltage': 'voltage_v', 'v': 'voltage_v',
            'current': 'current_a', 'i': 'current_a',
            'temperature': 'temperature_c', 'temp': 'temperature_c', 'celltemp': 'temperature_c',
            'discharge_capacity_ah': 'discharge_capacity_ah', 'q_discharge': 'discharge_capacity_ah',
            'charge_capacity_ah': 'charge_capacity_ah', 'q_charge': 'charge_capacity_ah'
        }
        df.rename(columns={k:v for k,v in rename_map.items() if k in df.columns}, inplace=True)

        # 2) Numeric conversion & cleanup
        df = df.apply(pd.to_numeric, errors='coerce')
        df = df.dropna(axis=1, how='all').dropna(how='all')
        df = df.drop_duplicates()

        # 3) Advanced interpolation (group-aware for NCA)
        num_cols = df.select_dtypes(np.number).columns
        if len(num_cols) > 0 and len(df) > 1:
            time_col = next((c for c in ['test_time_s', 'time_s'] if c in df), None)
            cycle_col = next((c for c in ['cycle_index', 'cycle'] if c in df), None)

            # Sort and interpolate per cycle
            if cycle_col and time_col:
                df = df.sort_values([cycle_col, time_col])
                df[num_cols] = df.groupby(cycle_col)[num_cols].transform(
                    lambda x: x.interpolate(method='linear', limit=CONFIG['interp_gap'])
                )
            else:
                if time_col:
                    df = df.sort_values(time_col)
                df[num_cols] = df[num_cols].interpolate(method='linear', limit=CONFIG['interp_gap'])

            # Forward/backward fill remaining
            df[num_cols] = df[num_cols].ffill().bfill()
            df = df.dropna(subset=num_cols)

        # 4) NCA-specific physics (SNL high-voltage NCA)[web:73]
        if 'voltage_v' in df:
            df = df[(df['voltage_v'] >= CONFIG['v_min']) & (df['voltage_v'] <= CONFIG['v_max'])]

        # Capacity limits (SNL NCA ~2.5-3Ah max)
        cap_cols = [c for c in df.columns if 'capacity_ah' in c or 'q_' in c]
        for col in cap_cols:
            if col in df:
                df[col] = df[col].clip(lower=0, upper=CONFIG['capacity_max'])

        # Non-negative for energy/power
        for col in num_cols:
            if any(x in col for x in ['energy_wh', 'power_w']):
                df[col] = df[col].clip(lower=0)

        # 5) Cycle validation (monotonic time)
        if CONFIG['drop_bad_cycles'] and cycle_col and time_col:
            is_valid = df.groupby(cycle_col)[time_col].apply(
                lambda x: x.is_monotonic_increasing
            )
            bad_cycles = is_valid[~is_valid].index
            if len(bad_cycles) > 0:
                df = df[~df[cycle_col].isin(bad_cycles)]
                print(f"    ⚠ Removed {len(bad_cycles)} non-monotonic cycles")

        # 6) Save if viable
        if len(df) < 10:
            print(f"    ⚠ Only {len(df)} rows after cleaning")
            return False

        out_name = f"{path.stem}_cleaned.csv"
        out_path = clean_dir / out_name
        df.to_csv(out_path, index=False)

        retention = len(df) / orig_rows * 100
        print(f"    ✅ {len(df):,} rows ({retention:.0f}% retention)")
        return True

    except Exception as e:
        print(f"    ❌ {type(e).__name__}: {str(e)[:60]}")
        return False

# Process all files
print("\n" + "="*70)
processed_files = 0
for i, csv_file in enumerate(csv_files):
    print(f"File {i+1}/{len(csv_files)}: ", end="")
    if clean_snl_nca_file(csv_file):
        processed_files += 1

print(f"\n{'='*70}")
print(f"🎉 SNL NCA CLEANING COMPLETE!")
print(f"✅ Processed: {processed_files}/{len(csv_files)} files")

# Step 7: Create final ZIP
final_zip = "/content/snl_nca_battery_cleaned.zip"
with zipfile.ZipFile(final_zip, 'w', zipfile.ZIP_DEFLATED) as zf:
    for cleaned_csv in clean_dir.glob("*.csv"):
        zf.write(cleaned_csv, cleaned_csv.name)

print(f"\n📦 Download: {final_zip}")
!du -sh {final_zip} {clean_dir}

# Validation peek
if list(clean_dir.glob("*.csv")):
    sample_df = pd.read_csv(list(clean_dir.glob("*.csv"))[0])
    print(f"\n📊 SAMPLE ({list(clean_dir.glob('*.csv'))[0].name}):")
    print(f"Shape: {sample_df.shape}")
    print(f"Columns: {list(sample_df.columns)}")
    if 'voltage_v' in sample_df:
        print(f"Voltage: {sample_df['voltage_v'].min():.2f}V → {sample_df['voltage_v'].max():.2f}V ✓")
    print(f"Missing values: {sample_df.isnull().sum().sum()}")


📤 Upload your SNL NCA battery dataset ZIP:


Saving SNL NCA-20260122T061440Z-1-001.zip to SNL NCA-20260122T061440Z-1-001.zip
✅ Processing: SNL NCA-20260122T061440Z-1-001.zip
✅ Extracted to /content/snl_nca_raw
Found 48 CSV files

File 1/48:   Cleaning SNL_18650_NCA_35C_0-100_0.5-1C_a_cycle_data.csv...
    ✅ 514 rows (100% retention)
File 2/48:   Cleaning SNL_18650_NCA_25C_20-80_0.5-0.5C_c_timeseries.csv...
    ✅ 288,664 rows (100% retention)
File 3/48:   Cleaning SNL_18650_NCA_25C_20-80_0.5-0.5C_a_timeseries.csv...
    ✅ 271,689 rows (100% retention)
File 4/48:   Cleaning SNL_18650_NCA_25C_40-60_0.5-0.5C_a_timeseries.csv...
    ✅ 469,560 rows (100% retention)
File 5/48:   Cleaning SNL_18650_NCA_35C_0-100_0.5-1C_c_cycle_data.csv...
    ✅ 514 rows (100% retention)
File 6/48:   Cleaning SNL_18650_NCA_35C_0-100_0.5-2C_a_timeseries.csv...
    ✅ 80,233 rows (100% retention)
File 7/48:   Cleaning SNL_18650_NCA_35C_0-100_0.5-1C_a_timeseries.csv...
    ✅ 65,270 rows (100% retention)
File 8/48:   Cleaning SNL_18650_NCA_25C_20-80_0.5-0.5C_c